# Transformando Longitude e Latitude em UTM
---

### Utilizando o Pyproj
---

In [1]:
import pandas as pd 
import numpy as np
from utils import *
from pyproj import Transformer

In [2]:
trans = Transformer.from_crs(
    "epsg:4326",
    "+proj=utm +zone=22 +ellps=WGS84 +south=True", # Zona 22 = Região do estado do Paraná, BR | 'south=True' para usar o false northing, pois estamos abaixo da linha do equador
    always_xy=True,
)

In [3]:
df_cidades = pd.read_csv('data/dados_hist_clima_concatenados/INMET_S_PR_MEDIA_OUTUBRO_01-01-2006_A_30-11-2021.CSV', index_col=[0])

In [4]:
xx, yy = trans.transform(df_cidades["LONGITUDE"].values, df_cidades["LATITUDE"].values)
df_cidades["X"] = xx
df_cidades["Y"] = yy

In [5]:
# Removendo exponencial para deixar 6 casas decimais na UTM
df_cidades['Y'] = pd.to_numeric(df_cidades['Y'].apply(lambda x: '%.6f' % x))

In [6]:
df_cidades.head(2)

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),...,UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",CIDADE,LATITUDE,LONGITUDE,X,Y
DATA (YYYY-MM-DD),,,,,,,,,,,,,,,,,,,,,
2006-10-01,0.18,916.47,916.71,916.26,981.89,13.98,10.66,14.32,13.63,10.97,...,79.79,81.73,112.24,6.05,2.77,"CURITIBA, Parana, Brazil",-25.429596,-49.271272,673850.722132,7.186356e+06
2006-10-08,0.06,911.53,911.82,911.28,1072.84,18.04,14.20,18.60,17.58,14.61,...,76.85,79.47,132.42,5.35,2.33,"CURITIBA, Parana, Brazil",-25.429596,-49.271272,673850.722132,7.186356e+06


In [ ]:
# df_cidades.to_csv(f'data/dados_hist_clima_concatenados/INMET_S_PR_MEDIA_UTM_01-01-2006_A_30-11-2021.CSV')

### Utilizando o Shapely
---

In [7]:
import geopandas as gpd
from shapely.geometry import Point

In [14]:
df_cidades['geometry'] = df_cidades.apply(lambda row: Point(row.LONGITUDE, row.LATITUDE), axis=1)

In [15]:
# change to GeoDataFrame
geodf = gpd.GeoDataFrame(df_cidades)
geodf.crs= {'init': 'epsg:4326'}

/home/laryssastephanie/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [16]:
# change the projection of geodf
geodf = geodf.to_crs("+proj=utm +zone=22 +ellps=WGS84 +datum=WGS84 +units=m +no_def +south=True")

In [17]:
# Inserindo resultados em colunas x e y
geodf["X"] = geodf.geometry.apply(lambda row:row.x)
geodf["Y"] = geodf.geometry.apply(lambda row:row.y)

In [18]:
# Removendo exponencial para deixar 6 casas decimais na UTM
geodf['Y'] = geodf['Y'].apply(lambda x: '%.6f' % x)

In [19]:
geodf.head()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),...,"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",CIDADE,LATITUDE,LONGITUDE,X,Y,geometry
DATA (YYYY-MM-DD),,,,,,,,,,,,,,,,,,,,,
2006-10-01,0.18,916.47,916.71,916.26,981.89,13.98,10.66,14.32,13.63,10.97,...,81.73,112.24,6.05,2.77,"CURITIBA, Parana, Brazil",-25.429596,-49.271272,673850.722132,7186355.862258,POINT (673850.722 7186355.862)
2006-10-08,0.06,911.53,911.82,911.28,1072.84,18.04,14.20,18.60,17.58,14.61,...,79.47,132.42,5.35,2.33,"CURITIBA, Parana, Brazil",-25.429596,-49.271272,673850.722132,7186355.862258,POINT (673850.722 7186355.862)
2006-10-15,0.11,911.87,912.15,911.65,1091.36,18.68,14.81,19.15,18.18,15.22,...,79.88,103.19,5.22,2.36,"CURITIBA, Parana, Brazil",-25.429596,-49.271272,673850.722132,7186355.862258,POINT (673850.722 7186355.862)
2006-10-22,0.07,913.04,913.29,912.82,967.55,15.70,12.25,16.14,15.27,12.64,...,81.06,105.03,5.87,2.73,"CURITIBA, Parana, Brazil",-25.429596,-49.271272,673850.722132,7186355.862258,POINT (673850.722 7186355.862)
2006-10-29,0.00,912.23,912.51,912.01,1428.66,18.79,12.67,19.46,18.12,13.19,...,70.42,123.96,6.02,2.73,"CURITIBA, Parana, Brazil",-25.429596,-49.271272,673850.722132,7186355.862258,POINT (673850.722 7186355.862)
